## Setup

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

import sys, os, yaml

DATASET = "Churn"
COLAB = 'google.colab' in sys.modules
DEBUG = False
SEED = 666

In [4]:
COLAB = 'google.colab' in sys.modules

if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
  if not os.path.isdir(ROOT): os.makedirs(ROOT)
else:
  ROOT = "./"

def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output']: makedirs(d)

## Load Dataset

In [5]:
df = pd.read_pickle(f"{ROOT}/data/data.pkl")
print(df.shape)
df.head(2)

(7032, 21)


,CustomerID,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,Female,No,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,2,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   CustomerID        7032 non-null   int64   
 1   Gender            7032 non-null   category
 2   SeniorCitizen     7032 non-null   category
 3   Partner           7032 non-null   category
 4   Dependents        7032 non-null   category
 5   Tenure            7032 non-null   int64   
 6   PhoneService      7032 non-null   category
 7   MultipleLines     7032 non-null   category
 8   InternetService   7032 non-null   category
 9   OnlineSecurity    7032 non-null   category
 10  OnlineBackup      7032 non-null   category
 11  DeviceProtection  7032 non-null   category
 12  TechSupport       7032 non-null   category
 13  StreamingTV       7032 non-null   category
 14  StreamingMovies   7032 non-null   category
 15  Contract          7032 non-null   category
 16  PaperlessBilling  7032 n

## Preprocessing Dataset

### Identify target and features

In [7]:
target = "Churn"
print(f"target = {target}")

cat_features = [c for c in df.select_dtypes("category").columns if c!= target]
print(f"\nCategorical features ({len(cat_features)}): {cat_features}")

num_features = [c for c in df.select_dtypes(["int","float"]).columns if c!= target]
print(f"\nNumerical features ({len(num_features)}): {num_features}")



target = Churn

Categorical features (16): ['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

Numerical features (4): ['CustomerID', 'Tenure', 'MonthlyCharges', 'TotalCharges']


### Train/Test split

* Split data train/test -> this time (60% train/40% test)

In [8]:
df.Churn.value_counts(normalize=True)

No     0.734215
Yes    0.265785
Name: Churn, dtype: float64

* `train_size` -> % must use float / num_of_rows must use int
* `randon_state` -> 
* `stratify` -> divide most equally the rate of yes/no between train and test

In [9]:
from sklearn.model_selection import train_test_split

df_train,df_test = train_test_split(df, train_size=0.60, stratify=df[target], random_state=SEED)

print(df_train.shape,df_test.shape)

(4219, 21) (2813, 21)


In [10]:
df_train.Churn.value_counts(normalize=True)

No     0.734297
Yes    0.265703
Name: Churn, dtype: float64

In [11]:
df_test.Churn.value_counts(normalize=True)

No     0.734092
Yes    0.265908
Name: Churn, dtype: float64

### Encode target

* `LabelEncoder` takes a column with strings (categorical) and change to numerical by a Map

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

le.fit(df_train[target])

y_train = le.transform(df_train[target])
y_test = le.transform(df_test[target])

In [13]:
df_train[target].head()

193      No
6108     No
3580    Yes
2195     No
3809     No
Name: Churn, dtype: category
Categories (2, object): ['No', 'Yes']

In [14]:
y_train[:5]

array([0, 0, 1, 0, 0])

In [15]:
le.inverse_transform(y_train[:5])

array(['No', 'No', 'Yes', 'No', 'No'], dtype=object)

### Encode Categorical features

In [16]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

ohe.fit(df_train[cat_features])

X_cat_train = ohe.transform(df_train[cat_features])
X_cat_test = ohe.transform(df_test[cat_features])

df_cat_train = pd.DataFrame(X_cat_train.toarray(), columns=ohe.get_feature_names_out())
df_cat_test = pd.DataFrame(X_cat_test.toarray(), columns=ohe.get_feature_names_out())
print(df_cat_train.shape, df_cat_test.shape)

df_cat_train.head(2)

(4219, 36) (2813, 36)


,Gender_Female,Gender_Male,SeniorCitizen_No,SeniorCitizen_Yes,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


### Scale / Transform numerical features

In [17]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

ss.fit(df_train[num_features])

X_num_train = ss.transform(df_train[num_features])
X_num_test = ss.transform(df_test[num_features])

df_num_train = pd.DataFrame(X_num_train, columns=ss.get_feature_names_out())
df_num_test = pd.DataFrame(X_num_test, columns=ss.get_feature_names_out())
print(df_num_train.shape, df_num_test.shape)

df_num_train.head(2)



(4219, 4) (2813, 4)


,CustomerID,Tenure,MonthlyCharges,TotalCharges
0,-1.640247,1.597275,1.487874,2.557942
1,1.267948,-0.596193,-1.323820,-0.804777


### Construct dataframe for model features

In [18]:
df_model_train = pd.concat([df_cat_train, df_num_train], axis=1)
df_model_test = pd.concat([df_cat_test, df_num_test], axis=1)
print(df_model_train.shape, df_model_test.shape)

(4219, 40) (2813, 40)


## Model Selection

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

classifiers = {
    "KNN" : KNeighborsClassifier(),
    "KNN(3)" : KNeighborsClassifier(3),
    "DT" : DecisionTreeClassifier(),
    "DT(max_depth=5)" : DecisionTreeClassifier(max_depth=5),
    "LR" : LogisticRegression(max_iter=1000),
    "RF" : RandomForestClassifier(),
    "AdaBoost" : AdaBoostClassifier(),
}

In [20]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

metrics = {
    'accuracy' : accuracy_score,
    'precision' : precision_score,
    'recall' : recall_score,
    'f1' : f1_score,
    'roc_auc' : roc_auc_score,
}

In [21]:
data = []
for model_name, model in classifiers.items():
    row = {'Model': model_name}
    model.fit(df_model_train, y_train)
    
    for metric_name, metric in metrics.items():
        # Scoring on SEEN data - effectively "useless"
        y_pred = model.predict(df_model_train)
        row['train_'+metric_name] = metric(y_train, y_pred)
    
        # Scoring on UNSEEN data - important
        y_pred = model.predict(df_model_test)
        row['test_'+metric_name] = metric(y_test, y_pred)
    data.append(row)
    #print(f"{name:20s} accuracy\ttrain = {train_accuracy:.2%} \ttest = {test_accuracy:.2%}")
df_results = pd.DataFrame(data)

In [22]:
df_results.head()

,Model,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f1,test_f1,train_roc_auc,test_roc_auc
0,KNN,0.835032,0.756843,0.700661,0.541775,0.661909,0.554813,0.680734,0.548217,0.779792,0.692419
1,KNN(3),0.863238,0.746534,0.750460,0.522124,0.727029,0.552139,0.738559,0.536712,0.819777,0.684544
2,DT,1.000000,0.728404,1.000000,0.489276,1.000000,0.487968,1.000000,0.488621,1.000000,0.651732
3,DT(max_depth=5),0.802797,0.787416,0.622561,0.589286,0.654773,0.661765,0.638261,0.623426,0.755566,0.747347
4,LR,0.803745,0.804124,0.659066,0.650382,0.541481,0.569519,0.594515,0.607270,0.720063,0.729311


In [34]:
def highlight_col(x):
    g = 'background-color: green'
    b = 'background-color: blue'
    p = 'background-color: pink'

    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 0] = g
    df1.iloc[:, 1] = b

    return df1 
df_results.style.apply(highlight_col, axis=None)

,Model,train_accuracy,test_accuracy,train_precision,test_precision,train_recall,test_recall,train_f1,test_f1,train_roc_auc,test_roc_auc
0,KNN,0.835032,0.756843,0.700661,0.541775,0.661909,0.554813,0.680734,0.548217,0.779792,0.692419
1,KNN(3),0.863238,0.746534,0.750460,0.522124,0.727029,0.552139,0.738559,0.536712,0.819777,0.684544
2,DT,1.000000,0.728404,1.000000,0.489276,1.000000,0.487968,1.000000,0.488621,1.000000,0.651732
3,DT(max_depth=5),0.802797,0.787416,0.622561,0.589286,0.654773,0.661765,0.638261,0.623426,0.755566,0.747347
4,LR,0.803745,0.804124,0.659066,0.650382,0.541481,0.569519,0.594515,0.607270,0.720063,0.729311
5,RF,1.000000,0.788127,1.000000,0.628378,1.000000,0.497326,1.000000,0.555224,1.000000,0.695394
6,AdaBoost,0.808959,0.800213,0.671382,0.648562,0.550401,0.542781,0.604902,0.590975,0.726460,0.718122


In [24]:
from sklearn.metrics import confusion_matrix

def business_benefit_score(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    # return [tn, fp, fn, tp] -> true negative, false positive, false negative, true positive

    benefits = np.array([1000, b, c, d]) 
    #abcd => values to be multiplied to tn, fp, fn, tp
    
    return (benefits*cm).sum()/cm.sum()

In [25]:
from sklearn.model_selection import cross_val_score

for name, model in classifiers.items():
    
    scores = cross_val_score(model, df_model_train, y_train, cv=10)
    
    print(f"{name:20s} accuracy\tCV= {scores.mean():.2%} \tstd = {scores.std():.2%}")

KNN                  accuracy	CV= 76.16% 	std = 2.03%
KNN(3)               accuracy	CV= 74.61% 	std = 1.70%
DT                   accuracy	CV= 72.27% 	std = 1.81%
DT(max_depth=5)      accuracy	CV= 78.19% 	std = 1.17%
LR                   accuracy	CV= 80.07% 	std = 1.69%
RF                   accuracy	CV= 79.07% 	std = 1.59%


KeyboardInterrupt: ignored